<a href="https://colab.research.google.com/github/chouhandiksha/bigdataproject/blob/colab/notebooks/analyse_LA_mobility_poverty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Spark SQL Documentation:** 
https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

In [6]:
!pip install ipython-autotime

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.83 s (started: 2021-04-04 21:56:07 +00:00)


In [7]:
# Install required dependancies
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 72kB/s 
     |████████████████████████████████| 204kB 31.4MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=89cd0ae57e3df5ed93688faed28563969c9c15e82a6708cd8e3c240e389078b2
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 30 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [8]:
# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

time: 48.1 ms (started: 2021-04-04 21:57:09 +00:00)


In [9]:
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

time: 314 ms (started: 2021-04-04 21:57:10 +00:00)


In [10]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

time: 4.92 s (started: 2021-04-04 21:57:10 +00:00)


In [11]:
# Mount drive with data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 18.7 s (started: 2021-04-04 21:57:15 +00:00)


In [12]:
# Demographic Data Chicago
!ls drive/MyDrive/big-data-project/data/clean-data/la/poverty_and_white_percentage.csv

!ls drive/MyDrive/big-data-project/data/clean-data/la/social/2020/



drive/MyDrive/big-data-project/data/clean-data/la/poverty_and_white_percentage.csv
2020-01-01-social-distancing.csv  2020-07-02-social-distancing.csv
2020-01-02-social-distancing.csv  2020-07-03-social-distancing.csv
2020-01-03-social-distancing.csv  2020-07-04-social-distancing.csv
2020-01-04-social-distancing.csv  2020-07-05-social-distancing.csv
2020-01-05-social-distancing.csv  2020-07-06-social-distancing.csv
2020-01-06-social-distancing.csv  2020-07-07-social-distancing.csv
2020-01-07-social-distancing.csv  2020-07-08-social-distancing.csv
2020-01-08-social-distancing.csv  2020-07-09-social-distancing.csv
2020-01-09-social-distancing.csv  2020-07-10-social-distancing.csv
2020-01-10-social-distancing.csv  2020-07-11-social-distancing.csv
2020-01-11-social-distancing.csv  2020-07-12-social-distancing.csv
2020-01-12-social-distancing.csv  2020-07-13-social-distancing.csv
2020-01-13-social-distancing.csv  2020-07-14-social-distancing.csv
2020-01-14-social-distancing.csv  2020-07-15-s

In [13]:
# Set path to data folder
path = Path('drive/MyDrive/big-data-project/data/clean-data')
city = 'la'

time: 2.88 ms (started: 2021-04-04 21:57:41 +00:00)


In [14]:
# Read data into dataframe
df_soc = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'social/2020/*.csv'))
df_soc.show()

+---+------------+--------------------+--------------------+------------+---------------------------+--------------------------+------------------------------------------+----------------------------+----------------------+------------------------+--------------------+-------------------------------+-------------------------------+--------------------+-------------------------+--------------------------+----------------------+----------------------------+---------------------------+-----------------------------+
|_c0|         cbg|    date_range_start|      date_range_end|device_count|distance_traveled_from_home|bucketed_distance_traveled|median_dwell_at_bucketed_distance_traveled|completely_home_device_count|median_home_dwell_time|bucketed_home_dwell_time|at_home_by_each_hour|part_time_work_behavior_devices|full_time_work_behavior_devices|    destination_cbgs|delivery_behavior_devices|median_non_home_dwell_time|candidate_device_count|bucketed_away_from_home_time|median_percentage_time_

In [15]:
# View schema
df_soc.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- cbg: string (nullable = true)
 |-- date_range_start: string (nullable = true)
 |-- date_range_end: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- distance_traveled_from_home: string (nullable = true)
 |-- bucketed_distance_traveled: string (nullable = true)
 |-- median_dwell_at_bucketed_distance_traveled: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- median_home_dwell_time: string (nullable = true)
 |-- bucketed_home_dwell_time: string (nullable = true)
 |-- at_home_by_each_hour: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)
 |-- destination_cbgs: string (nullable = true)
 |-- delivery_behavior_devices: string (nullable = true)
 |-- median_non_home_dwell_time: string (nullable = true)
 |-- candidate_device_count: string (nullable = true)
 |-- bucketed_away_from_home_ti

In [16]:
# # Take small sample of data to experiment with
# df_soc = df_soc.limit(100)
# df_soc.show()

time: 2.72 ms (started: 2021-04-04 21:58:00 +00:00)


In [17]:
df_soc.createOrReplaceTempView('T')
df_soc = spark.sql('SELECT cbg, date_range_start, device_count, completely_home_device_count FROM T')
df_soc.show()

+------------+--------------------+------------+----------------------------+
|         cbg|    date_range_start|device_count|completely_home_device_count|
+------------+--------------------+------------+----------------------------+
|060371831013|2020-02-07T00:00:...|          45|                           6|
|060374034022|2020-02-07T00:00:...|         155|                          24|
|060375326041|2020-02-07T00:00:...|          31|                           4|
|060375544031|2020-02-07T00:00:...|         130|                          27|
|060375718003|2020-02-07T00:00:...|          35|                          10|
|060590018024|2020-02-07T00:00:...|          38|                          13|
|060590320463|2020-02-07T00:00:...|         184|                          24|
|060590626281|2020-02-07T00:00:...|         108|                          32|
|060371374011|2020-02-07T00:00:...|          72|                          30|
|060372398012|2020-02-07T00:00:...|          41|                

In [18]:
# View schema
df_soc.printSchema()

root
 |-- cbg: string (nullable = true)
 |-- date_range_start: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)

time: 5.77 ms (started: 2021-04-04 21:58:01 +00:00)


In [19]:
# Add completely home percentage column
df_soc = df_soc.withColumn('completely_home_percentage', (df_soc['completely_home_device_count']/df_soc['device_count']) * 100)
df_soc.show()

+------------+--------------------+------------+----------------------------+--------------------------+
|         cbg|    date_range_start|device_count|completely_home_device_count|completely_home_percentage|
+------------+--------------------+------------+----------------------------+--------------------------+
|060371831013|2020-02-07T00:00:...|          45|                           6|        13.333333333333334|
|060374034022|2020-02-07T00:00:...|         155|                          24|        15.483870967741936|
|060375326041|2020-02-07T00:00:...|          31|                           4|        12.903225806451612|
|060375544031|2020-02-07T00:00:...|         130|                          27|         20.76923076923077|
|060375718003|2020-02-07T00:00:...|          35|                          10|         28.57142857142857|
|060590018024|2020-02-07T00:00:...|          38|                          13|         34.21052631578947|
|060590320463|2020-02-07T00:00:...|         184|       

In [20]:
df_soc.printSchema()

root
 |-- cbg: string (nullable = true)
 |-- date_range_start: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- completely_home_percentage: double (nullable = true)

time: 8.84 ms (started: 2021-04-04 21:58:01 +00:00)


In [21]:
df_soc.createOrReplaceTempView('t')
# get mean percentage for each cbg
df_soc = spark.sql('SELECT cbg, AVG(completely_home_percentage) AS mean_completely_home_percentage FROM t GROUP BY cbg')
df_soc.createOrReplaceTempView('t')
df_soc.show()

+------------+-------------------------------+
|         cbg|mean_completely_home_percentage|
+------------+-------------------------------+
|060371042031|               32.8148808521497|
|060374034051|             39.991210434957296|
|060590758091|             27.629728783069826|
|060372123031|              37.22036369480373|
|060590744051|             29.707895435772926|
|060371958032|             31.128491756584676|
|060372145021|              38.43423278432839|
|060590525061|              31.85805623501183|
|060376021032|                35.118237618907|
|060372122022|              38.17613330725545|
|060375743006|              28.61729276768516|
|060372948101|              36.96847880310482|
|060372198001|             28.060332029036132|
|060371244001|              32.90643517770641|
|060375720012|              30.15167856792122|
|060375701002|              32.82995806467224|
|060375401014|              31.25585333401366|
|060375540023|              33.27982253792674|
|060373107021

In [22]:
# Read poverty data
# Read data into dataframe
df_pov = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'poverty_and_white_percentage.csv'))
df_pov.createOrReplaceTempView('d')
df_pov.show()

+------------+---------+------------------+------------------+
|         cbg|pop_total|poverty_percentage|    perc_whiteonly|
+------------+---------+------------------+------------------+
|060373104003|     1286|17.884914463452567| 69.51788491446345|
|060590865023|     2545|22.789783889980352| 85.34381139489194|
|060376510024|     1285|               0.0|42.490272373540854|
|060376511011|     2708| 7.745398773006134| 50.70162481536189|
|060590994023|      672|12.969283276450511| 38.54166666666667|
|060590995094|      342| 17.83625730994152| 77.48538011695906|
|060376511012|     2483|2.6178010471204187|50.302053966975436|
|060376512011|      859|2.8846153846153846| 82.42142025611176|
|060376512012|     1179| 1.441899915182358|  74.1306191687871|
|060376512014|      989|2.4266936299292214| 70.57633973710819|
|060376512211|     3373|  9.95684340320592| 40.61666172546695|
|060590995092|      938| 5.756929637526652|  84.9680170575693|
|060590995095|      576|14.583333333333334|           7

In [23]:
# join mobility and poverty
result = spark.sql('SELECT d.*, t.mean_completely_home_percentage FROM t INNER JOIN d ON t.cbg = d.cbg')
result.createOrReplaceTempView('t')
result.show()

+------------+---------+------------------+------------------+-------------------------------+
|         cbg|pop_total|poverty_percentage|    perc_whiteonly|mean_completely_home_percentage|
+------------+---------+------------------+------------------+-------------------------------+
|060371042031|     2577|  24.3892828999212| 57.00426852929763|               32.8148808521497|
|060374034051|     1696| 1.474056603773585| 23.99764150943396|             39.991210434957296|
|060590758091|     1025| 7.512195121951219| 66.63414634146342|             27.629728783069826|
|060372123031|     3420| 37.28963684676705| 19.12280701754386|              37.22036369480373|
|060590744051|     1752| 29.54944411936805| 73.63013698630137|             29.707895435772926|
|060371958032|     1196| 28.26086956521739| 70.81939799331104|             31.128491756584676|
|060372145021|     2135|27.775175644028106| 54.05152224824356|              38.43423278432839|
|060590525061|      881| 8.286038592508513| 79.001

In [24]:
# Count number of rows
spark.sql('SELECT COUNT(cbg) FROM t').show()

+----------+
|count(cbg)|
+----------+
|      8243|
+----------+

time: 1min 38s (started: 2021-04-04 22:07:08 +00:00)


In [25]:
# sort by poverty
result = spark.sql('SELECT * FROM t ORDER BY CAST(poverty_percentage AS float) DESC')
result.createOrReplaceTempView('t')
result.show()

+------------+---------+------------------+------------------+-------------------------------+
|         cbg|pop_total|poverty_percentage|    perc_whiteonly|mean_completely_home_percentage|
+------------+---------+------------------+------------------+-------------------------------+
|060372227001|     4056| 88.46960167714884| 47.68244575936883|              20.07742122256767|
|060372247001|     2886| 87.87222523010287| 53.46500346500347|              30.32778196266256|
|060372426002|     2289|  87.1559633027523|20.882481432940146|              30.31784431986003|
|060590745013|     1282|  86.1154446177847| 44.46177847113885|             34.458428177656714|
|060372063002|     2173| 85.41862652869237|28.071790151863784|                42.563030569054|
|060372063001|     1768|   85.122089721749| 25.16968325791855|              42.56110440252481|
|060590626262|     1506| 83.59893758300133| 29.61487383798141|             25.276351701588343|
|060372653041|     1921| 81.87869822485207| 50.598

In [26]:
# group by poverty range


time: 1.86 ms (started: 2021-04-04 22:10:28 +00:00)


In [27]:
result_df = result.toPandas()
result_df

,cbg,pop_total,poverty_percentage,perc_whiteonly,mean_completely_home_percentage
0,060372227001,4056,88.46960167714884,47.68244575936883,20.077421
1,060372247001,2886,87.87222523010287,53.46500346500347,30.327782
2,060372426002,2289,87.1559633027523,20.882481432940146,30.317844
3,060590745013,1282,86.1154446177847,44.46177847113885,34.458428
4,060372063002,2173,85.41862652869237,28.071790151863784,42.563031
...,...,...,...,...,...
8238,060374802023,858,0.0,62.93706293706294,36.195509
8239,060371417003,489,0.0,95.0920245398773,25.338658
8240,060376023022,571,0.0,52.71453590192644,28.423306
8241,060372771002,725,0.0,74.20689655172413,32.785335


time: 1min 44s (started: 2021-04-04 22:10:28 +00:00)


In [28]:
# plot bar graph
alt.Chart(result_df).mark_point().encode(
    alt.X('poverty_percentage:Q'),
    alt.Y('mean_completely_home_percentage:Q'),
    tooltip=['cbg','poverty_percentage','mean_completely_home_percentage']
).properties(width=400, height=400).interactive()

alt.Chart(...)

time: 341 ms (started: 2021-04-04 22:12:12 +00:00)


In [29]:
temp = result_df.groupby(pd.cut(result_df['poverty_percentage'].astype(float), np.arange(0,110, 10))).mean().reset_index()

time: 61.2 ms (started: 2021-04-04 22:12:12 +00:00)


In [30]:
temp['poverty_percentage'] = temp['poverty_percentage'].astype(str)

time: 3.8 ms (started: 2021-04-04 22:12:12 +00:00)


In [31]:
temp['color'] = ['#45a0d1' if x>30 else '#ff4833' for x in temp['mean_completely_home_percentage']]
temp

,poverty_percentage,mean_completely_home_percentage,color
0,"(0, 10]",33.332056,#45a0d1
1,"(10, 20]",34.585032,#45a0d1
2,"(20, 30]",34.918295,#45a0d1
3,"(30, 40]",35.210405,#45a0d1
4,"(40, 50]",35.010863,#45a0d1
5,"(50, 60]",34.638087,#45a0d1
6,"(60, 70]",33.698850,#45a0d1
7,"(70, 80]",34.354756,#45a0d1
8,"(80, 90]",30.746520,#45a0d1
9,"(90, 100]",NaN,#ff4833


time: 22 ms (started: 2021-04-04 22:12:12 +00:00)


In [32]:
# 
alt.Chart(temp).mark_point().encode(
    alt.X('poverty_percentage:N'),
    alt.Y('mean_completely_home_percentage'),
    color = alt.Color('color',scale=None),
    tooltip=[alt.Tooltip('poverty_percentage'),
             alt.Tooltip('mean_completely_home_percentage')]
).properties(width=400, height=400).interactive()

alt.Chart(...)

time: 29.5 ms (started: 2021-04-04 22:12:12 +00:00)


In [33]:
result_df['poverty_percentage'].astype(float).dtype

dtype('float64')

time: 9.41 ms (started: 2021-04-04 22:12:13 +00:00)
